# Send Plots to Zooniverse

First we do bad things to install the necessary packages

In [5]:
#!pip install --user panoptes-client
#!pip install python-dotenv

In [1]:
%load_ext dotenv
%dotenv

In [2]:
# put a text file in this directory named .env with contents
# export ZOONIVERSE_USER="your_email@address.com"
# export ZOONIVERSE_PASSWORD="hunter2"

In [3]:
import os
from glob import glob
from panoptes_client import Panoptes, Project, SubjectSet, Subject
from astropy.utils.console import ProgressBar

In [4]:
client = Panoptes.connect(username=os.environ.get('ZOONIVERSE_USER'), password=os.environ.get('ZOONIVERSE_PASSWORD'))

I already made the project through the web UI

In [5]:
project = Project.find(slug='ebellm/ztf-dippers')

Directory where the plots are:

In [6]:
project.links

In [7]:
print([(ss.id, ss.display_name) for ss in project.links.subject_sets])

[]


In [8]:
#indir = '/epyc/users/kyboone/ztf_boyajian/zooniverse/'
indir = '/astro/users/keatonb/ZTF_Boyajian/Candidate_plots/'

In [9]:
# delete old ones if desired
#subject_set = SubjectSet.find(80863)
#for subject in subject_set.subjects:
#    subject_set.remove(subject.id)

In [10]:
for workflow in project.links.workflows:
    print(workflow.display_name)

Scan Dippers 2021
Scan Dippers


In [15]:
# make a new subject set
subject_set = SubjectSet()

subject_set.links.project = project
subject_set.display_name = 'ZTF_dippers_2021'

subject_set.save()

{'subject_sets': [{'id': '96097',
   'display_name': 'ZTF_dippers_2021',
   'set_member_subjects_count': 0,
   'metadata': {},
   'created_at': '2021-06-25T16:46:02.302Z',
   'updated_at': '2021-06-25T16:46:02.302Z',
   'href': '/subject_sets/96097',
   'links': {'project': '12979', 'workflows': [], 'subjects': []}}],
 'links': {'subject_sets.project': {'href': '/projects/{subject_sets.project}',
   'type': 'projects'},
  'subject_sets.workflows': {'href': '/workflows?subject_set_id={subject_sets.id}',
   'type': 'workflows'}},
 'meta': {'subject_sets': {'page': 1,
   'page_size': 20,
   'count': 1,
   'include': [],
   'page_count': 1,
   'previous_page': None,
   'next_page': None,
   'first_href': '/subject_sets',
   'previous_href': None,
   'next_href': None,
   'last_href': '/subject_sets'}}}

In [16]:
project.reload()

In [17]:
files = glob(f'{indir}/*.png')

In [18]:
len(files)

596

In [19]:
def get_id(filename):
    return int(filename.split('/')[-1].split('.')[0])

In [20]:
get_id(files[0])

164653156082208171

In [21]:
new_subjects = []

with ProgressBar(len(files), ipython_widget=True) as bar:

    for filename in files:
        subject = Subject()

        subject.links.project = project
        subject.add_location(filename)

        subject.metadata.update({'ps1_id':get_id(filename)})

        subject.save()
        new_subjects.append(subject)
        bar.update()
subject_set.add(new_subjects)

FloatProgress(value=0.0)

In [22]:
subject_set.add(new_subjects)

In [23]:
workflow = project.links.workflows[0]
workflow.links.subject_sets.add(subject_set)